# Analysis of the performance of different ways to concatenate strings in java

Java Strings are immutable (ish if you are willing to do some terrible dirty things), so concatenation is a slow operation that ends up creating an entirely new string object. Java uses a string builder under the hood to perform this operation. This means that multiple concatenations in a loop will cause a lot of unnecessary objects to be created that will need to be garbage collected.


## Naive approach
For control, this function just uses the + operator to concatenate strings together.
```java
//Basic Concat
public static String testFunc(String num) {
    for (int i = 0; i < CONCATS; i++) {
        num = num + ".";
    }
    return num;
}
```

```java
public class FastStringAppender {

    private static StringBuilder sb = new StringBuilder();

    public static synchronized String concat(String[] args) {
        sb.setLength(0);
        for (String arg : args) {
            sb.append(arg);
        }
        return sb.toString();
    }
}

public static String testFunc2(String num) {
    String [] arr = new String[CONCATS + 1];
    Arrays.fill(arr, ".");
    arr[0] = num;
    return FastStringAppender.concat(arr);
}
```

```java
//Static String Builder
private static final StringBuilder sb = new StringBuilder();
public static String testFunc3(String num) {
    sb.setLength(0);
    sb.append(num);
    for (int i = 0; i < CONCATS; i++) {
        sb.append(".");
    }
    return sb.toString();
}
```

```java
//Static String Buffer
private static final StringBuffer stringBuffer = new StringBuffer();
public static String testFunc4(String num) {
    stringBuffer.setLength(0);
    stringBuffer.append(num);
    for (int i = 0; i < CONCATS; i++) {
        stringBuffer.append(".");
    }
    return stringBuffer.toString();
}
```

```java
//String Buffer Non-Static
public static String testFunc5(String num) {
    StringBuilder stringBuffer = new StringBuilder();
    stringBuffer.append(num);
    for (int i = 0; i < CONCATS; i++) {
        stringBuffer.append(".");
    }
    return stringBuffer.toString();
}
```

```java
//String Builder passed from function
public static String testFunc6(String num, StringBuilder stringBuilder) {
    stringBuilder.setLength(0);
    stringBuilder.append(num);
    for (int i = 0; i < CONCATS; i++) {
        stringBuilder.append(".");
    }
    return stringBuilder.toString();
}
```

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ipywidgets import interact, Dropdown
from IPython.display import display

df = pd.read_csv("data/JavaStringConcatenationBenchmarkingData.csv")

df['ErrorRate'] = df['Error'] / df['OPS']
df['Effective OPS'] = df['OPS'] * df['Iterations'] * df['Concatenations']

functions = np.unique(df['Function'].values)
iterations = np.unique(df['Iterations'].values)
concatenations = np.unique(df['Concatenations'].values)
jvm = np.unique(df['JVM'].values)
computer = np.unique(df['Computer'].values)
optionChoices = ['Compare Functions', 'Compare Iterations', 'Compare Concatenations']

optionsDropdown = Dropdown(options=optionChoices)
option1Dropdown = Dropdown()
option2Dropdown = Dropdown()

functionDropdown = Dropdown(options=functions, description='Wow')
iterationDropdown = Dropdown(options=iterations)
concatenationDropdown = Dropdown(options=concatenations)

@interact(options=optionsDropdown,
          option1=option1Dropdown,
          option2=option2Dropdown)
def wow(options, option1, option2):
    #Set options
    if options == optionChoices[0]:
        option1Dropdown.options = iterations
        option1Dropdown.description = 'Iterations'
        option2Dropdown.options = concatenations
        option2Dropdown.description = 'Concat'
    elif options == optionChoices[1]:
        option1Dropdown.options = functions
        option1Dropdown.description = 'Functions'
        option2Dropdown.options = concatenations
        option2Dropdown.description = 'Concat'
    elif options == optionChoices[2]:
        option1Dropdown.options = functions
        option1Dropdown.description = 'Functions'
        option2Dropdown.options = iterations
        option2Dropdown.description = 'Iterations'

    #Set Default Values
    if options is None:
        optionsDropdown.value = optionsDropdown.options[0]
        option1Dropdown.value = option1Dropdown.options[0]
        option2Dropdown.value = option2Dropdown.options[0]
    else:
        if option1 is None:
            option1Dropdown.value = option1Dropdown.options[0]
        if option2 is None:
            option2Dropdown.value = option2Dropdown.options[0]

    #Plot Graph
    if options == optionChoices[0]:
        filtered_df = df[df['Iterations'] == option1]
        filtered_df = filtered_df[filtered_df['Concatenations'] == option2]
        plt.bar(filtered_df['Function'], filtered_df['Effective OPS'] )
        plt.show()

    elif options == optionChoices[1]:
        for function in functions:
            filtered_df = df[df['Function'] == function]
            filtered_df = filtered_df[filtered_df['Concatenations'] == option2]
            graph = plt.plot(filtered_df['Iterations'], filtered_df['Effective OPS'], label=function)
            graph[0].axes.set_xscale('log')
        plt.legend()
        plt.show()

    elif options == optionChoices[2]:
        for function in functions:
            filtered_df = df[df['Function'] == function]
            filtered_df = filtered_df[filtered_df['Iterations'] == option2]
            graph = plt.plot(filtered_df['Concatenations'], filtered_df['Effective OPS'], label=function)
            graph[0].axes.set_xscale('log')
        plt.legend()
        plt.show()


interactive(children=(Dropdown(description='options', options=('Compare Functions', 'Compare Iterations', 'Com…